In [1]:
import fiftyone.zoo as foz


dataset = foz.load_zoo_dataset("quickstart")
print(dataset)

NumExpr defaulting to 8 threads.
Dataset already downloaded
Loading existing dataset 'quickstart'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use
Name:        quickstart
Media type:  image
Num samples: 200
Persistent:  False
Tags:        ['validation']
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    uniqueness:   fiftyone.core.fields.FloatField
    predictions:  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)


In [6]:
sample = dataset.first()

In [54]:
import numpy as np


def detections_to_shapes(detections, frame_size, metadata=None):
    """Convert a list of detections into a shapes tuple.
    
    Parameters
    ----------
    detections : fiftyone.core.labels.Detection

    Returns
    -------
    tuple
        Shape layer data tuple, consisting of list of shapes
        layer data, shapes layer metadata, and `Shapes`.
    """
    polylines = detections.to_polylines().polylines
    shapes_data = []
    properties = {}
    for polyline in polylines:
        polyline_dict = polyline.to_dict()
        points = np.array(polyline_dict.pop('points')[0])[:, ::-1]
        points = np.multiply(points, frame_size)
        shapes_data.append(points)
        for key, value in polyline_dict.items():
            if not isinstance(value, dict) and not isinstance(value, list):
                existing_values = properties.get(key, [])
                existing_values.append(value)
                properties[key] = existing_values
    layer_metadata = {'properties': properties, 'shape_type': 'polygon'}
    if metadata is not None:
        layer_metadata.update(metadata)

    return shapes_data, layer_metadata, 'Shapes'

In [72]:
from imageio import imread


def sample_to_image(sample):
    image = imread(sample.filepath)
    metadata = {}
    metadata['name'] = sample.id
    return (image, metadata, 'Image')


def sample_to_layers(sample):
    layer_data = []
    image_data = sample_to_image(sample)
    frame_size = image_data[0].shape[:-1]
    layer_data.append(image_data)
    metadata = {'name': 'ground_truth',
                'edge_width':2,
                'edge_color': 'green',
                'face_color':'transparent',
                'text': {
                    'text': 'label',
                    'size': 12,
                    'color': 'green',
                    'anchor': 'upper_left',
                    'translation': [-1, 0],
                    }
                }
    layer_data.append(detections_to_shapes(sample.ground_truth, frame_size, metadata))
    metadata = {'name': 'predictions',
                'edge_width':2,
                'edge_color': 'red',
                'face_color':'transparent',
                'text': {
                    'text': 'label',
                    'size': 12,
                    'color': 'red',
                    'anchor': 'upper_left',
                    'translation': [-1, 0],
                    }
                }
    layer_data.append(detections_to_shapes(sample.predictions, frame_size, metadata))
    return layer_data

In [80]:
# image = sample_to_image(sample)
# shapes = detections_to_shapes(sample.ground_truth, image[0].shape[:-1])

layer_data = sample_to_layers(dataset.take(1).first())

In [81]:
for ld in layer_data:
    viewer._add_layer_from_data(*ld)

In [89]:
import napari
viewer = napari.Viewer()

In [82]:
samples = list(dataset.iter_samples())

In [90]:
from qtpy.QtWidgets import QListWidget


def load_sample(idx):
    idx = int(idx.text())
    print('Loading', idx, '...')
    sample = samples[idx]
    viewer.layers.select_all()
    viewer.layers.remove_selected()
    layer_data = sample_to_layers(sample)    
    for ld in layer_data:
        viewer._add_layer_from_data(*ld)
    print('... done.')

list_widget = QListWidget()
for n in range(len(samples)):
    list_widget.addItem(str(n))

list_widget.currentItemChanged.connect(load_sample)

viewer.window.add_dock_widget(list_widget, area='right')

Loading <PyQt5.QtWidgets.QListWidgetItem object at 0x7faf764c3eb0> ...
Loading <PyQt5.QtWidgets.QListWidgetItem object at 0x7faf764c37d0> ...
Loading <PyQt5.QtWidgets.QListWidgetItem object at 0x7faf7429fcd0> ...


In [91]:
1

1

Loading <PyQt5.QtWidgets.QListWidgetItem object at 0x7faf764c3b90> ...
